# Import packages / libraries

In [1]:
pip install py-solc-x

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import the Web3 object and establish an HTTP connection
# See also: https://medium.com/validitylabs/how-to-interact-with-the-ethereum-blockchain-and-create-a-database-with-python-and-sql-3dcbd579b3c0
from IPython.display import display, HTML , Image
from ipywidgets import Dropdown
# from PIL import Image
from web3 import Web3 
import requests
import getpass
import json
from solcx import compile_standard, install_solc

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [142]:
# Connecting to the UZHETH Blockchain
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))

# Connection test
print(web3.isConnected())

True


# Get Started 

## Get Accounts

In [176]:
print(web3.eth.accounts)
print("balance 1:", web3.fromWei(web3.eth.get_balance(web3.eth.accounts[0]), "ether"), "ether")
print("balance 2:", web3.fromWei(web3.eth.get_balance(web3.eth.accounts[1]), "ether"), "ether")

web3.eth.default_account = web3.eth.accounts[0]

print(web3.eth.default_account)

['0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26', '0x24174b610C359f4a4c099117e8aDB15E270C7cC0']
balance 1: 27.937144086 ether
balance 2: 20 ether
0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26


In [177]:
def handle_change():
    print(users_dropdown.value)

def new_user(number):
    description = "User " + number + "\t"
    users_dropdown = Dropdown(description=description, options=list_users)
    users_dropdown.on_trait_change(handle_change, name="value")
    display(users_dropdown)
    
list_users = web3.eth.accounts
print("Here you can select the users to swap tokens.")


# Get Dropdown selection and select two different users
new_user("1")
selected_user_1 = users_dropdown.value

new_user("2")
selected_user_2 = users_dropdown.value

Here you can select the users to swap tokens.


Dropdown(description='User 1\t', options=('0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26', '0x24174b610C359f4a4c0…

Dropdown(description='User 2\t', options=('0xF5ba136D777A2843b01A5a2A6d844a944Fb3cc26', '0x24174b610C359f4a4c0…

In [178]:
#read abi
# Interract with Smart contracts
# see also: https://medium.com/deepyr/interacting-with-ethereum-using-web3-py-and-jupyter-notebooks-e4207afa0085

# ABI code of the contract; Stored in a file (copied from Remix)
tokenswap_abi_path = "Programming/22_HS/PancakeSwap/TokenSwap.abi"
obj_file = open(tokenswap_abi_path, "r")
swap_abi = obj_file.read()

# ABI code of the contract; Stored in a file (copied from Remix)
token_abi_path = "Programming/22_HS/PancakeSwap/PancakeToken.abi"
obj_file = open(token_abi_path, "r")
token_abi = obj_file.read()

with open("Programming/22_HS/PancakeSwap/Swap.json") as s:
    swap_json = json.load(s)
    swap_bytecode = swap_json["object"]

with open("Programming/22_HS/PancakeSwap/Token.json") as t:
    token_json = json.load(t)
    token_bytecode = token_json["object"]

In [179]:
swap_contract = web3.eth.contract(abi=swap_abi, bytecode=swap_bytecode)

In [180]:
pancake_contract = web3.eth.contract(abi=token_abi, bytecode=token_bytecode)

In [157]:
uzh_contract = web3.eth.contract(abi=token_abi, bytecode=token_bytecode)

In [167]:
print("Type in the password of user 1")
password_1 = getpass.getpass() 

print("Type in the password of user 2")
password_2 = getpass.getpass()

Type in the password of user 1


 ········


Type in the password of user 2


 ········


In [198]:
def unlock(account, password):
    web3.geth.personal.unlock_account(account, password)
unlock(selected_user_1, password_1)
unlock(selected_user_2, password_2)

In [199]:
# make pancake and uzh tokens on the accounts of both users
tx_p = pancake_contract.constructor("Pancake Token", "Cake").transact({'from': selected_user_1})
tx_u = uzh_contract.constructor("UZH Token", "UZHETH").transact({'from': selected_user_2})

In [183]:
tx_receipt_p = web3.eth.wait_for_transaction_receipt(tx_p)
tx_receipt_p

AttributeDict({'blockHash': HexBytes('0xc8968e97f1c08e89d86dd566a3eea82f645d6c5b8db9294bcea6cbe5a5ccdcf7'),
 'blockNumber': 3788176,
 'contractAddress': '0x598CADF941Ad911034eE7fb49f9f9347f49d895F',
 'cumulativeGasUsed': 1181813,
 'effectiveGasPrice': 1000000000,
 'from': '0x24174b610C359f4a4c099117e8aDB15E270C7cC0',
 'gasUsed': 1181813,
 'logs': [AttributeDict({'address': '0x598CADF941Ad911034eE7fb49f9f9347f49d895F',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000000000000000000000024174b610c359f4a4c099117e8adb15e270c7cc0')],
   'data': '0x0000000000000000000000000000000000000000000000056bc75e2d63100000',
   'blockNumber': 3788176,
   'transactionHash': HexBytes('0xada60c8eddb20b7fc5b64cb28ece33d0bc69503c318e8d1710b97ae711584a09'),
   'transactionIndex': 0,
   'blockHash': HexBytes('0xc8968e97f1c08e89d86dd566a3eea82f645d6c5b8db9294bcea

In [184]:
tx_receipt_u = web3.eth.wait_for_transaction_receipt(tx_u)
tx_receipt_u

AttributeDict({'blockHash': HexBytes('0xc8968e97f1c08e89d86dd566a3eea82f645d6c5b8db9294bcea6cbe5a5ccdcf7'),
 'blockNumber': 3788176,
 'contractAddress': '0x324862dA26D119C5d66DB106ce71F3d72F9361a5',
 'cumulativeGasUsed': 2363602,
 'effectiveGasPrice': 1000000000,
 'from': '0x24174b610C359f4a4c099117e8aDB15E270C7cC0',
 'gasUsed': 1181789,
 'logs': [AttributeDict({'address': '0x324862dA26D119C5d66DB106ce71F3d72F9361a5',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000000000000000000000024174b610c359f4a4c099117e8adb15e270c7cc0')],
   'data': '0x0000000000000000000000000000000000000000000000056bc75e2d63100000',
   'blockNumber': 3788176,
   'transactionHash': HexBytes('0x75d46623b4f9a8231cded4ad34d737ce3c34f322468d58e5a2515883ffd13c99'),
   'transactionIndex': 1,
   'blockHash': HexBytes('0xc8968e97f1c08e89d86dd566a3eea82f645d6c5b8db9294bcea

In [185]:
cake_address = tx_receipt_p.contractAddress
pancake_token = web3.eth.contract(cake_address, abi=token_abi)
print("Current Amount Cake Token:\t",pancake_token.functions.balanceOf(selected_user_1).call())
print("Cake Address:", cake_address)

uzh_address = tx_receipt_u.contractAddress
uzh_token = web3.eth.contract(uzh_address, abi=token_abi)
print("Current Amount UZH ETH Token:\t", uzh_token.functions.balanceOf(selected_user_2).call())
print("UZH ETH Token Address:", uzh_address)
#swap = web3.eth.contract(tx_receipt_s.contractAddress, abi=swap_abi)

Current Amount Cake Token:	 100000000000000000000
Cake Address: 0x598CADF941Ad911034eE7fb49f9f9347f49d895F
Current Amount UZH ETH Token:	 100000000000000000000
UZH ETH Token Address: 0x324862dA26D119C5d66DB106ce71F3d72F9361a5


In [200]:
unlock(selected_user_1, password_1)
tx_s = swap_contract.constructor(cake_address, selected_user_1, uzh_address, selected_user_2).transact({
    'from': selected_user_1, 'gas': 420000, 'gasPrice': 1000000000})
print(tx_s)

b'\xf7\xddL@\xc7\xb2H\r\x88\x82\x8f\x81\xae\x02\x81\xdc\xf7\xcfV\xfdE\xcd\x83n\t\x95\xadbW\xa8\xe08'


In [188]:
tx_receipt_s = web3.eth.wait_for_transaction_receipt(tx_s)
print(tx_receipt_s)
swap = web3.eth.contract(tx_receipt_s.contractAddress, abi=swap_abi)
swap_address = swap.address
print("\n\nSwap Address:",swap_address)

AttributeDict({'blockHash': HexBytes('0x639af5f0f3b222c14fc2b9f888aa8e2d1191cf2cb5017b429cc3439b9418b0ce'), 'blockNumber': 3788179, 'contractAddress': '0x7F0B802faF19057811eF7eEC6d52f67F8A8F5dc3', 'cumulativeGasUsed': 420000, 'effectiveGasPrice': 1000000000, 'from': '0x24174b610C359f4a4c099117e8aDB15E270C7cC0', 'gasUsed': 420000, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 0, 'to': None, 'transactionHash': HexBytes('0x4ec3510612e8754029bb4477165c1df9f155f8b41b2f05d9e8db8ad2393

In [189]:
amount_p = int(input("How much Cake Tokens do you want to swap?"))
amount_u = int(input("How much UZH ETH Tokens do you want to swap?"))

How much Cake Tokens do you want to swap? 20
How much UZH ETH Tokens do you want to swap? 30


In [225]:
unlock(selected_user_1, password_1)
unlock(selected_user_2, password_2)
print("Accounts unlocked.")

print("Approve Pancake Token Swap") 
approve_p = pancake_token.functions.approve(swap_address, amount_p * 10**18).transact({'from': selected_user_1})
approve_p_receipt = web3.eth.wait_for_transaction_receipt(approve_p)
print("Current Amount Cake Token allowed to swap:\t",pancake_token.functions.allowance(selected_user_1, swap_address).call())

print("Approve UZH ETH Token Swap")
approve_u = uzh_token.functions.approve(swap_address, amount_u * 10**18).transact({'from': selected_user_2})
approve_u_receipt = web3.eth.wait_for_transaction_receipt(approve_u)
print("Current Amount UZH ETH Token allowed to swap:\t",uzh_token.functions.allowance(selected_user_2, swap_address).call())

Accounts unlocked.
Current Amount Cake Token allowed to swap:	 20000000000000000000
Current Amount UZH ETH Token allowed to swap:	 30000000000000000000


In [226]:
print("You want to swap:", amount_p, "Cake Tokens with", amount_u, "UZH ETH Tokens")

You want to swap: 20 Cake Tokens with 30 UZH ETH Tokens


In [232]:
unlock(selected_user_1, password_1)
unlock(selected_user_2, password_2)
print("Accounts unlocked.")

tx_swap_tok = swap.functions.swap(amount_p * 10**18, amount_u * 10**18).transact({'from':selected_user_1, 'to':selected_user_2})
tx_receipt_swap_tok = web3.eth.wait_for_transaction_receipt(tx_swap_tok)
tx_receipt_swap_tok

Accounts unlocked.


AttributeDict({'blockHash': HexBytes('0x4b2b4a2d6beeb7930833009f3c7b3b7428f5c5bacc322fc36db5441ab8c18dcd'),
 'blockNumber': 3788300,
 'contractAddress': None,
 'cumulativeGasUsed': 21488,
 'effectiveGasPrice': 1000000000,
 'from': '0x24174b610C359f4a4c099117e8aDB15E270C7cC0',
 'gasUsed': 21488,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0x24174b610C359f4a4c099117e8aDB15E270C7cC0',
 'transactionHash': HexBytes('0x32fe4d4fc8aa4975a111df06ecfc7b69527a537448a0693728

In [233]:
print("Current Amount Cake Token:\t",pancake_token.functions.balanceOf(selected_user_1).call())
print("Current Amount UZH ETH Token:\t",pancake_token.functions.balanceOf(selected_user_2).call())

Current Amount Cake Token:	 100000000000000000000
Current Amount UZH ETH Token:	 100000000000000000000
